In [1]:
a='this is a test'

import utils as u



In [3]:
df = u.read_pickle('port-new-analysis-sp500.p')

In [4]:
df.head(1)

,ticker,rank,close,atr,cls_gt_ma,gap,num_shares,cost,pct_alloc
309,MKTX,4.01012,519.070007,18.922971,True,False,0.052846,27.430682,4.346328
